In [ ]:
import os
import openai
import sys
sys.path.append('../..')

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.environ['OPENAI_API_KEY']
KEY  = os.environ['JINA_EMBEDDING_KEY']


In [ ]:
from langchain.document_loaders import PyPDFLoader

# Load PDF
loaders = [
    # Duplicate documents on purpose - messy data
    PyPDFLoader("documents/temp.pdf"),
    PyPDFLoader("documents/lm741.pdf")
]
docs = []
for loader in loaders:
    docs.extend(loader.load())

In [ ]:
# Split
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1500,
    chunk_overlap = 150
)

In [ ]:
splits = text_splitter.split_documents(docs)
len(splits)

In [ ]:
from langchain.embeddings import JinaEmbeddings
embedding = JinaEmbeddings(jina_api_key = KEY, model_name="jina-embeddings-v2-base-en")


In [ ]:
from langchain.vectorstores import Chroma
persist_directory = 'docs/chroma1/'
vectordb = Chroma.from_documents(
    documents=splits,
    embedding=embedding,
    persist_directory=persist_directory
)

In [ ]:
print(vectordb._collection.count())

In [ ]:
question = "is there an email i can ask for help"
docs = vectordb.similarity_search(question,k=3)
len(docs)

In [ ]:
vectordb.persist()

In [ ]:
import datetime
current_date = datetime.datetime.now().date()
if current_date < datetime.date(2023, 9, 2):
    llm_name = "gpt-3.5-turbo-0301"
else:
    llm_name = "gpt-3.5-turbo"
print(llm_name)

In [ ]:
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(model_name=llm_name, temperature=0)

In [ ]:
from langchain.chains import RetrievalQA

In [ ]:
# qa_chain = RetrievalQA.from_chain_type(
#     llm,
#     retriever=vectordb.as_retriever()
# )
# from langchain.prompts import PromptTemplate


# question="how many words are there?"
# result = qa_chain({"query": question})
# result['result']

In [ ]:
# from langchain.prompts import PromptTemplate

# # Build prompt
# template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer. Use three sentences maximum. Keep the answer as concise as possible. Always say "thanks for asking!" at the end of the answer. 
# {context}
# Question: {question}
# Helpful Answer:"""
# QA_CHAIN_PROMPT = PromptTemplate.from_template(template)


In [ ]:
# # Run chain
# qa_chain = RetrievalQA.from_chain_type(
#     llm,
#     retriever=vectordb.as_retriever(),
#     return_source_documents=True,
#     chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}
# )

In [ ]:

# Build prompt
from langchain.prompts import PromptTemplate

# template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer. Use three sentences maximum. Keep the answer as concise as possible. Always say "Fuck you !" at the end of the answer. 
template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer. Use three sentences maximum. Keep the answer as concise as possible. Must say "Fuck You!" at the end of the answer. 
{context}
Question: {question}
Helpful Answer:"""
QA_CHAIN_PROMPT = PromptTemplate(
    input_variables=["context", "question"],
    template=template,
)

# Run chain
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(),
    return_source_documents=True,
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT},
)

question = "Is probability a class topic?"
result = qa_chain({"query": question})
result["result"]


# ### Memory

from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)


# ### ConversationalRetrievalChain


from langchain.chains import ConversationalRetrievalChain

retriever = vectordb.as_retriever()
qa = ConversationalRetrievalChain.from_llm(llm, retriever=retriever, memory=memory)


In [ ]:
chat_history = []
answer = ""
db_query = ""
db_response = []

In [ ]:
query ="hi?"

result = qa({"question": query, "chat_history": chat_history})
chat_history.extend([(query, result["answer"])])
# db_query = result["generated_question"]
# db_response = result["source_documents"]
answer = result["answer"]
print(answer)